In [2]:
import logging
import warnings
from matplotlib import MatplotlibDeprecationWarning
from pyspark.sql import SparkSession
from ydata_profiling import ProfileReport
from ydata_profiling.config import Settings

logging.basicConfig(level=logging.INFO)

# Local paths for input and output
input_path = "/output/cleaned_datasets/part-00000-84b4bb66-a373-431b-8655-e13205cafff1-c000.csv"
output_path = "cleaned_titles_report.html"

spark_session = (
    SparkSession.builder
    .appName("SparkProfiling")
    .master("local[*]")
    .config("spark.executor.memory", "4g")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

spark_df = spark_session.read.csv(input_path, header=True, inferSchema=True)

# Configuration settings for ydata_profiling
cfg = Settings()
cfg.infer_dtypes = False
cfg.correlations["auto"].calculate = False
cfg.correlations["pearson"].calculate = False
cfg.correlations["spearman"].calculate = False
cfg.interactions.continuous = False
cfg.missing_diagrams["bar"] = False
cfg.missing_diagrams["heatmap"] = False
cfg.missing_diagrams["matrix"] = False
cfg.samples.tail = 0
cfg.samples.random = 0

# Create and start the monitoring process
warnings.filterwarnings("ignore", category=MatplotlibDeprecationWarning)

# ydata_profiling tool
profile = ProfileReport(spark_df, config=cfg)

profile.to_file(output_path)

print(f"Profile report saved to {output_path}")


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/07/04 18:45:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

24/07/04 18:53:14 WARN CacheManager: Asked to cache already cached data.) / 2]  
24/07/04 18:53:17 WARN CacheManager: Asked to cache already cached data.        
24/07/04 18:53:22 WARN CacheManager: Asked to cache already cached data.        
24/07/04 18:53:22 WARN CacheManager: Asked to cache already cached data.) / 2]
24/07/04 18:53:24 WARN CacheManager: Asked to cache already cached data.        
24/07/04 18:53:31 WARN CacheManager: Asked to cache already cached data.) / 2]  
24/07/04 18:55:03 WARN CacheManager: Asked to cache already cached data.1) / 10]
24/07/04 18:55:12 WARN CacheManager: Asked to cache already cached data.) / 52] 
24/07/04 18:56:23 WARN CacheManager: Asked to cache already cached data.        
INFO:py4j.clientserver:Closing down clientserver connection                     
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.clientserver:Closing down clientserver connection
INFO:py4j.cl

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Profile report saved to cleaned_titles_report.html
